In [ ]:
import requests
import time
import pandas as pd
import os

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
url = "https://ows.goszakup.gov.kz/v3/subject/all"
filename = "goszakup_subjAll_data.csv"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

params = {"limit": 500}  
all_data = []

# Проверяем, есть ли уже сохраненные данные
if os.path.exists(filename):
    df_existing = pd.read_csv(filename, encoding="utf-8-sig", sep=";")
    last_update_date = df_existing["last_update_date"].max()  # Определяем последнюю обновленную запись
    print(f"Последняя дата обновления: {last_update_date}")
else:
    last_update_date = None

while True:
    try:
        if last_update_date:
            params.update({"last_update_date": last_update_date})

        response = requests.get(url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            print("Новых данных нет, загрузка завершена.")
            break

        # Фильтруем только новые записи
        new_items = [item for item in items if item["last_update_date"] > last_update_date]
        if not new_items:
            print("Нет новых записей.")
            break

        all_data.extend(new_items)
        print(f"Загружено {len(all_data)} новых записей...")

        # Обновляем дату последнего обновления
        last_update_date = max(item["last_update_date"] for item in new_items)

        # Промежуточное сохранение
        if len(all_data) % 1000 < 500:
            df_new = pd.DataFrame(all_data)
            df_new.to_csv(
                filename, 
                mode="a", 
                header=not os.path.exists(filename),  # Шапка только если файла нет
                index=False, 
                encoding="utf-8-sig", 
                sep=";",  
                quoting=1,  # csv.QUOTE_NONNUMERIC — оборачивает только строки в кавычки
                quotechar='"',
                escapechar="\\"
            )
            print("Промежуточное сохранение...")

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        print("Ждём 30 сек и пробуем снова...")
        time.sleep(30)

# Финальное сохранение
if all_data:
    df_new = pd.DataFrame(all_data)
    df_new.to_csv(
        filename, 
        mode="a", 
        header=not os.path.exists(filename), 
        index=False, 
        encoding="utf-8-sig", 
        sep=";",  
        quoting=1,  
        quotechar='"',
        escapechar="\\"
    )
    print("Дозагрузка завершена, новые данные добавлены в файл.")
else:
    print("Новых данных не найдено.")